#Geniphy.ai v3.0
### Image Generation with stable-diffusion-xl-base-1.0 + Adding refiners + Creating Coloring Book + Stickers + Sticker sheet

In [3]:
!pip install transformers diffusers accelerate huggingface_hub gradio

In [4]:
from diffusers import DiffusionPipeline

# downloading stabilityai/stable-diffusion-xl-base-1.0
model_sdxl = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0")

# downloading stabilityai/stable-diffusion-xl-refiner-1.0
refiner_sdxl = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0")

# downloading ColoringBookRedmond-V2
# model_sdxl = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0")
model_sdxl.load_lora_weights("artificialguybr/ColoringBookRedmond-V2")

# downloading StickerSheet-Lora
model_sdxl.load_lora_weights("Norod78/SDXL-StickerSheet-Lora")

# downloading StickersRedmond
model_sdxl.load_lora_weights("artificialguybr/StickersRedmond")

# downloading Logo Lora
model_sdxl.load_lora_weights("artificialguybr/LogoRedmond-LogoLoraForSDXL-V2")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

(…)ogoRedmondV2-Logo-LogoRedmAF.safetensors:   0%|          | 0.00/171M [00:00<?, ?B/s]

In [5]:
%cd /root/.cache/huggingface/hub
%ls

/root/.cache/huggingface/hub
models--artificialguybr--ColoringBookRedmond-V2/
models--artificialguybr--LogoRedmond-LogoLoraForSDXL-V2/
models--artificialguybr--StickersRedmond/
models--Norod78--SDXL-StickerSheet-Lora/
models--stabilityai--stable-diffusion-xl-base-1.0/
models--stabilityai--stable-diffusion-xl-refiner-1.0/
version_diffusers_cache.txt
version.txt


In [6]:
!mkdir /content/Models
!mkdir /content/Models/LoRas
!mv /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-xl-base-1.0 /content/Models
!mv /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-xl-refiner-1.0 /content/Models
!mv /root/.cache/huggingface/hub/models--artificialguybr--ColoringBookRedmond-V2 /content/Models/LoRas
!mv /root/.cache/huggingface/hub/models--artificialguybr--LogoRedmond-LogoLoraForSDXL-V2 /content/Models/LoRas
!mv /root/.cache/huggingface/hub/models--artificialguybr--StickersRedmond /content/Models/LoRas
!mv /root/.cache/huggingface/hub/models--Norod78--SDXL-StickerSheet-Lora /content/Models/LoRas
!cd /content/Models
!ls

version_diffusers_cache.txt  version.txt


##Text to Image Generation Using SDXL, Refiner.
##Adding coloring book pages feature with LoRa

In [7]:
import gradio as gr
from PIL import Image
import torch
from diffusers import DiffusionPipeline

###############################
# Image Generation SDXL Model
###############################

def load_model_image_sdxl(model_path_sdxl, model_path_sdxl_refiner=None, model_path_coloring_lora=None, model_path_sticker_lora=None,
                          model_path_stickersheet_lora=None, model_path_logo_lora=None, torch_dtype=torch.float16, use_safetensors=True):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    ##### Load Base Model
    base = DiffusionPipeline.from_pretrained(model_path_sdxl, torch_dtype=torch_dtype, use_safetensors=use_safetensors)

    ##### Load Coloring LoRa Model
    if model_path_coloring_lora:
      base.load_lora_weights(model_path_coloring_lora)

    base.enable_model_cpu_offload()

    ##### Load Sticker LoRa Model
    if model_path_sticker_lora:
      base.load_lora_weights(model_path_sticker_lora)

    base.enable_model_cpu_offload()

    ##### Load Sticker Sheet LoRa Model
    if model_path_stickersheet_lora:
      base.load_lora_weights(model_path_stickersheet_lora)

    ##### Load Logo LoRa Model
    if model_path_logo_lora:
      base.load_lora_weights(model_path_logo_lora)

    base.enable_model_cpu_offload()

    ##### Load Refiner Model
    refiner = None
    if model_path_sdxl_refiner:
        refiner = DiffusionPipeline.from_pretrained(model_path_sdxl_refiner, text_encoder_2=base.text_encoder_2,
                                                    vae=base.vae, torch_dtype=torch_dtype, use_safetensors=use_safetensors)

        refiner.enable_model_cpu_offload()
        return base, refiner

    else:
        return base, None


def generate_image_sdxl(base, prompt, refiner=None, num_inference_steps=30, guidance_scale=7.5, high_noise_frac=0.6,
                        output_type="latent", verbose=False, temperature=0.65):

    if refiner:
        image = base(prompt=prompt, num_inference_steps=num_inference_steps, denoising_end=24, output_type=output_type,
                     verbose=verbose, temperature=temperature, guidance_scale=guidance_scale).images

        image = refiner(prompt=prompt, num_inference_steps=num_inference_steps, denoising_end=16, image=image,
                        verbose=verbose).images[0]

    else:
        image = base(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]

    return image


# Model paths
model_path_sdxl = "/content/Models/models--stabilityai--stable-diffusion-xl-base-1.0/snapshots/462165984030d82259a11f4367a4eed129e94a7b"
model_path_sdxl_refiner = "/content/Models/models--stabilityai--stable-diffusion-xl-refiner-1.0/snapshots/5d4cfe854c9a9a87939ff3653551c2b3c99a4356"
model_path_coloring_lora = "/content/Models/LoRas/models--artificialguybr--ColoringBookRedmond-V2/snapshots/c486ec1307a5dbf41c743392559ead0c82bdb614"
model_path_sticker_lora = "/content/Models/LoRas/models--artificialguybr--StickersRedmond/snapshots/f37c12ece066ef1fb0367abddac41c07d017cde4"
model_path_stickersheet_lora = "/content/Models/LoRas/models--Norod78--SDXL-StickerSheet-Lora/snapshots/18cb1c4cb7bc007ec6b6935dab27280f2e401b8a"
model_path_logo_lora = "/content/Models/LoRas/models--artificialguybr--LogoRedmond-LogoLoraForSDXL-V2/snapshots/8dd804b93747318e75063d49c29593d5deba736e"

# Load models
base, refiner = load_model_image_sdxl(model_path_sdxl, model_path_sdxl_refiner, model_path_coloring_lora, model_path_sticker_lora, model_path_stickersheet_lora)
coloring_lora_trigger = ", ColoringBookAF, Coloring Book"
sticker_lora_trigger = ", Stickers, Sticker"      # https://civitai.com/models/144142/stickersredmond-stickers-lora-for-sd-xl
stickersheet_lora_trigger = ", StickerSheet"      # https://civitai.com/models/137973/sdxl-sticker-sheet-lora
logo_lora_trigger = ", LogoRedmAF, Icons"          # https://civitai.com/models/124609/logoredmond-logo-lora-for-sd-xl-10


# Gradio interface
def generate_images(prompt, image_creation, coloring_book, sticker, stickersheet, logo):
    print("Generating image...")
    full_prompt = prompt
    if coloring_book:
        full_prompt += coloring_lora_trigger
    if sticker:
        full_prompt += sticker_lora_trigger
    if stickersheet:
        full_prompt += stickersheet_lora_trigger
    if logo:
        full_prompt += logo_lora_trigger
    if image_creation:
        image = generate_image_sdxl(base, full_prompt, refiner)
        return image
    else:
        return None

def clear_fields():
    return "", None, False, False  # Clear the prompt input and output

with gr.Blocks(css="""
    .orange-btn {background-color: orange; color: white;}
    .output-box {width: 900px; height: 300px; overflow: auto; object-fit: contain;}
    """) as iface:
    iface.title = "Geniphy.ai"
    iface.description = "Generate images based on your prompt using a diffusion model"

    with gr.Column():
        # Output area at the top with custom width
        output_display = gr.Image(label="Generated Image", elem_classes="output-box")

        # Prompt input field
        input_prompt = gr.Textbox(label="Enter Prompt", placeholder="Type prompt here or enter parameters", lines=2)

        # Checkboxes for image creation and coloring book
        with gr.Row():
            image_creation_checkbox = gr.Checkbox(label="Image Creation", value=True)
            coloring_book_checkbox = gr.Checkbox(label="Coloring Book", value=False)
            sticker_checkbox = gr.Checkbox(label="Sticker", value=False)
            stickersheet_checkbox = gr.Checkbox(label="Sticker Sheet", value=False)
            logo_checkbox = gr.Checkbox(label="Logo", value=False)

        # Buttons with custom layout
        with gr.Row():
            clear_button = gr.Button("Clear")
            submit_button = gr.Button("GENERATE", elem_classes="orange-btn")

        # Link buttons to functions
        submit_button.click(
            fn=generate_images,  # Call the image generation function
            inputs=[input_prompt, image_creation_checkbox, coloring_book_checkbox, sticker_checkbox, stickersheet_checkbox, logo_checkbox],
            outputs=output_display
        )

        clear_button.click(
            fn=clear_fields,
            inputs=None,
            outputs=[input_prompt, output_display, image_creation_checkbox, coloring_book_checkbox, sticker_checkbox, stickersheet_checkbox, logo_checkbox]
        )

iface.launch()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://55999d10bb42a14bd9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
